In [0]:
ServicePrincipalID = dbutils.secrets.get(scope="data-bricks", key="SPID")
ServicePrincipalKey = dbutils.secrets.get(scope="data-bricks", key="SPKey")
DirectoryID = dbutils.secrets.get(scope="data-bricks", key="DirectoryID")

Directory = f"https://login.microsoftonline.com/{DirectoryID}/oauth2/token"

container_name = 'my-csv'
storage_account_name = 'mainstorageaccountv2'

url=f'abfss://{container_name}@{storage_account_name}.dfs.core.windows.net'
mnt_path = f'/mnt/{container_name}'

configs = {
  "fs.azure.account.auth.type": "OAuth",
  "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
  "fs.azure.account.oauth2.client.id": ServicePrincipalID,
  "fs.azure.account.oauth2.client.secret": ServicePrincipalKey,
  "fs.azure.account.oauth2.client.endpoint": Directory,
  "fs.azure.createRemoteFileSystemDuringInitialization": "false"
}

dbutils.fs.mount(
  source = url,
  mount_point = mnt_path,
  extra_configs = configs
)

Out[1]: True

In [0]:
db_name = 'foo'
table_name = 'ufo'
obj_name = f'{db_name}.{table_name}'
obj_path = f'{mnt_path}/{obj_name}'

/mnt/my-csv/foo.ufo

In [0]:
dbutils.fs.rm(table_file_path, recurse=True)

In [0]:
spark.sql("""drop table if exists {}""".format(obj_name))

Out[11]: DataFrame[]

In [0]:
spark.sql("""drop database if exists {}""".format(db_name))

Out[12]: DataFrame[]

In [0]:
df = spark.read.format('csv').option("inferSchema", "true").option("header", "true").load("/mnt/my-csv/ufo.csv")
#df.show()

+------------+-----------+---------------+----------+-----+-----------+--------+
 LATITUDE| LONGITUDE| CITY|DATE_OCCUR|STATE| UFO_SHAPE|DURATION|
+------------+-----------+---------------+----------+-----+-----------+--------+
-95.78950112|36.05084018| Broken Arrow|11/22/2008| OK| Changing| 0|
-118.2453206|34.05349094| Los Angeles|04/08/2006| CA| Changing| 0|
-106.6486413|35.08418003| Albuquerque|05/26/2007| NM| Circle| 0|
-71.53660023|43.20725072| Concord|06/15/2001| NH| Circle| 0|
-76.51119034|43.45646037| Oswego|11/10/2005| NY| Circle| 0|
-118.4292502| 34.2608902| Pacoima|09/12/2013| CA| Circle| 0|
-82.30084031|39.80629006| Sommerset|11/22/2007| OH| Cylinder| 0|
-70.15095032|44.67026882| Farmington|12/07/2007| ME| Diamond| 0|
-86.28880059|36.20776105| Lebanon|09/08/2012| TN| Flash| 0|
-95.20141137|42.64746138| Storm Lake|08/27/2012| IA| Formation| 0|
-122.0309489|36.97402043| Santa Cruz|06/21/2011| CA| Light| 0|
-81.37739138|28.53823033| Orlando|08/10/2006| FL| Sphere| 0|
-83.63016087| 42.9274904| Grand Blanc|11/05/2005| MI|Unspecified| 0|
-104.9923007|39.74001085| Denver|12/31/2009| CO| Triangle| 0|
 -79.3849401|40.31664078| Latrobe|10/08/2012| PA| Unknown| 0|
-122.2452991|37.76683095| Alameda|11/29/2011| CA| Changing| 0|
-122.2526796| 38.1671203|American Canyon|01/18/2011| CA| Changing| 0|
-77.11396139|36.22885027| Aulander|09/03/2011| NC| Changing| 0|
-74.85727072|40.09762077| Bristol|08/13/2007| PA| Changing| 0|
-119.0507203|34.22289042| Camarillo|08/03/2004| CA| Changing| 0|
+------------+-----------+---------------+----------+-----+-----------+--------+
only showing top 20 rows

In [0]:
(df
 .write
 .format('delta')
 .option('mergeSchema','true')
 .save('/mnt/my-csv/output/foo.ufo')
)

In [0]:
spark.sql("""create table if not exists foo.ufo using delta location '/mnt/my-csv/output/foo.ufo'""")

Out[8]: DataFrame[]

In [0]:
dbutils.fs.unmount(mnt_path) 

/mnt/my-csv has been unmounted.
Out[13]: True